Exploring DSC SET Data

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [66]:
# load sets into dataframe
dsc_sets = pd.read_csv('datasets/dsc_sets.csv')
# merge duplicate columns
dsc_sets["Enrolled Resp Rate"] = dsc_sets["Enrolled/ Resp Rate"].fillna(dsc_sets["Enrolled/  Resp Rate"])
dsc_sets = dsc_sets.drop(columns=["Enrolled/ Resp Rate", "Enrolled/  Resp Rate"])
# remove Letter from Avg Grade Received
dsc_sets["Avg Grade Received"] = (
    dsc_sets["Avg Grade Received"]
    .str.extract(r'(\d+\.\d+|\d+)')   # extract first number
    .astype(float)
)
dsc_sets['Term'] = dsc_sets['Term'].str.lower()
# rename columns for easier access
dsc_sets.columns = dsc_sets.columns.str.replace('*', '', regex=False)
dsc_sets.columns = dsc_sets.columns.str.replace(' ', '_')
# delete unnecessary columns
dsc_sets = dsc_sets.drop(columns=["Course"])
# add required/
dsc_sets.head()

,Instructor,Term,Avg_Grade_Received,Avg_Hours_Worked,Learning_Average,Structure_Average,Environment_Average,course_title,Enrolled_Resp_Rate
0,"Bellur, Umesh",wi25,3.93,9.93,3.74,3.55,4.12,dsc180b,114 (14.04%)
1,"Bellur, Umesh",wi25,3.99,11.89,4.02,3.74,3.80,dsc180b,118 (16.95%)
2,"Rampure, Suraj",wi24,3.96,9.60,4.60,4.59,4.58,dsc180b,97 (10.31%)
3,"Rampure, Suraj",wi24,3.99,6.23,4.57,4.57,4.53,dsc180b,138 (9.42%)
4,"Weng, Tsui-Wei (lily)",sp25,3.71,7.38,4.19,4.13,4.24,dsc140b,124 (45.16%)


In [71]:
webreg_data = pd.read_csv('webreg_data/results/webreg_processed_data.csv')
webreg_data.head()
merged_data = webreg_data[['course', 'quarter', 'utilization_rate']]
merged_data = merged_data.copy()

#drop underscore from course column to match dsc_sets
merged_data['course'] = merged_data['course'].str.replace('_', '').str.lower()
merged_data = merged_data.rename(columns={'course': 'course_title', 'quarter': 'Term'})
final_data = pd.merge(dsc_sets, merged_data, on=['course_title', 'Term'], how='inner')
final_data.head()

,Instructor,Term,Avg_Grade_Received,Avg_Hours_Worked,Learning_Average,Structure_Average,Environment_Average,course_title,Enrolled_Resp_Rate,utilization_rate
0,"Bellur, Umesh",wi25,3.93,9.93,3.74,3.55,4.12,dsc180b,114 (14.04%),98.30
1,"Bellur, Umesh",wi25,3.99,11.89,4.02,3.74,3.80,dsc180b,118 (16.95%),98.30
2,"Weng, Tsui-Wei (lily)",sp25,3.71,7.38,4.19,4.13,4.24,dsc140b,124 (45.16%),97.62
3,"Huang, Biwei",wi25,3.53,6.38,4.25,4.27,4.57,dsc140b,42 (30.95%),29.66
4,"Salimi, Babak",sp25,3.63,5.44,4.04,3.61,4.05,dsc100,139 (74.10%),92.47


### Two-Way ANOVA: Division × Elective (on Learning Average)
Factors
- Division: Lower vs Upper
- Elective: Required vs Elective
- Interaction: Division × Elective